# Example 6: Automatic Parameter Selection
Firstly, let's import some modules that we will depend on in the next examples.

In [1]:
import time
import random
import pickle
import threading
import seal
from seal import ChooserEvaluator, \
	Ciphertext, \
	Decryptor, \
	Encryptor, \
	EncryptionParameters, \
	Evaluator, \
	IntegerEncoder, \
	KeyGenerator, \
	MemoryPoolHandle, \
	Plaintext, \
	SEALContext, \
	EvaluationKeys, \
	ChooserEncoder, \
	ChooserEvaluator, \
	ChooserPoly

SEAL contains an automatic parameter selection tool that can help the user
select optimal parameters that support a particular computation. In this
example we show how the tool can be used to find parameters for evaluating
the degree 3 polynomial 42x^3-27x+1 on an encrypted input encoded with the
IntegerEncoder. For this to be possible, we need to know an upper bound on
the size of the input, and in this example assume that x is an integer with
base-3 representation of length at most 10.

### Finding optimized parameters for computing 42x^3-27x+1:

The set of tools in the parameter selector are ChooserPoly, ChooserEvaluator,
ChooserEncoder, ChooserEncryptor, and ChooserDecryptor. Of these the most
important ones are ChooserPoly, which is an object representing the input
data both in plaintext and encrypted form, and ChooserEvaluator, which
simulates plaintext coefficient growth and noise budget consumption in the
computations. Here we use also the ChooserEncoder to conveniently obtain
ChooserPoly objects modeling the plaintext coefficients 42, -27, and 1.
Note that we are using the IntegerEncoder with base 3.

In [2]:
chooser_encoder = ChooserEncoder(3)
chooser_evaluator = ChooserEvaluator()

First we create a ChooserPoly representing the input data. You can think of
this modeling a freshly encrypted ciphertext of a plaintext polynomial of
length at most 10 coefficients, where the coefficients have absolute value
at most 1 (as is the case when using IntegerEncoder with base 3).

In [3]:
c_input = ChooserPoly(10, 1)

Normally Evaluator::exponentiate takes the evaluation keys as argument. Since
no keys exist here, we simply pass the desired decomposition bit count (15)
to the ChooserEvaluator::exponentiate function.
Here we compute the first term.

In [4]:
c_cubed_input = chooser_evaluator.exponentiate(c_input, 3, 15)
c_term1 = chooser_evaluator.multiply_plain(c_cubed_input, chooser_encoder.encode(42))

Then compute the second term.

In [5]:
c_term2 = chooser_evaluator.multiply_plain(c_input, chooser_encoder.encode(27))

Subtract the first two terms.

In [6]:
c_sum12 = chooser_evaluator.sub(c_term1, c_term2)

Finally add a plaintext constant 1.

In [7]:
c_result = chooser_evaluator.add_plain(c_sum12, chooser_encoder.encode(1))

The optimal parameters are now computed using the select_parameters
function in ChooserEvaluator. It is possible to give this function the
results of several distinct computations (as ChooserPoly objects), all
of which are supposed to be possible to perform with the resulting set
of parameters. However, here we have only one input ChooserPoly.

In [8]:
optimal_parms = EncryptionParameters()
chooser_evaluator.select_parameters([c_result], 0, optimal_parms)
print("Done")

Done


Create an SEALContext object for the returned parameters

In [9]:
def print_parameters(context):
    print("/ Encryption parameters:")
    print("| poly_modulus: " + context.poly_modulus().to_string())
    print("| coeff_modulus_size: " + (str)(context.total_coeff_modulus().significant_bit_count()) + " bits")
    print("| plain_modulus: " + (str)(context.plain_modulus().value()))
    print("| noise_standard_deviation: " + (str)(context.noise_standard_deviation()))
optimal_context = SEALContext(optimal_parms)
print_parameters(optimal_context)

/ Encryption parameters:
| poly_modulus: 1x^4096 + 1
| coeff_modulus_size: 110 bits
| plain_modulus: 512
| noise_standard_deviation: 3.19


Do the parameters actually make any sense? We can try to perform the
homomorphic computation using the given parameters and see what happens.

In [10]:
keygen = KeyGenerator(optimal_context)
public_key = keygen.public_key()
secret_key = keygen.secret_key()
ev_keys = EvaluationKeys()
keygen.generate_evaluation_keys(15, ev_keys)
encryptor = Encryptor(optimal_context, public_key)
evaluator = Evaluator(optimal_context)
decryptor = Decryptor(optimal_context, secret_key)
encoder = IntegerEncoder(optimal_context.plain_modulus(), 3)

Now perform the computations on some real data.

In [11]:
input_value = 12345
plain_input = encoder.encode(input_value)
print("Encoded " + (str)(input_value) + " as polynomial " + plain_input.to_string())
input_ciphertext = Ciphertext()
print("Encrypting: ")
encryptor.encrypt(plain_input, input_ciphertext)
print("Done")
print("Computing 42x^3-27x+1 on encrypted x=12345: ")
deg3_term = Ciphertext()
evaluator.exponentiate(input_ciphertext, 3, ev_keys, deg3_term)
evaluator.multiply_plain(deg3_term, encoder.encode(42))
deg1_term = Ciphertext()
evaluator.multiply_plain(input_ciphertext, encoder.encode(27), deg1_term)
evaluator.sub(deg3_term, deg1_term)
evaluator.add_plain(deg3_term, encoder.encode(1))
print("Done")

Encoded 12345 as polynomial 1x^9 + 1FFx^8 + 1FFx^6 + 1FFx^4 + 1x^3 + 1x^2 + 1FFx^1
Encrypting: 
Done
Computing 42x^3-27x+1 on encrypted x=12345: 
Done


Now deg3_term holds the result. We decrypt, decode, and print the result.

In [12]:
plain_result = Plaintext()
print("Decrypting: ")
decryptor.decrypt(deg3_term, plain_result)
print("Done")
print("Polynomial 42x^3-27x+1 evaluated at x=12345: " + (str)(encoder.decode_int64(plain_result)))

Decrypting: 
Done
Polynomial 42x^3-27x+1 evaluated at x=12345: 79017370138936


We should have a reasonable amount of noise room left if the parameter
selection was done properly. The user can experiment for instance by
changing the decomposition bit count, and observing how it affects the
result. Typically the budget should never be even close to 0. Instead,
SEAL uses heuristic upper bound estimates on the noise budget consumption,
which ensures that the computation will succeed with very high probability
with the selected parameters.

In [13]:
print("Noise budget in result: " + (str)(decryptor.invariant_noise_budget(deg3_term)) + " bits")

Noise budget in result: 46 bits
